## SUGD Experiments

This notebook runs multiple experiments with different configs files (```config_{i}.json``` under the ```configs/``` folder) and produces summary reports with the final performance metrics. It can be used for manual hyperparameter-tuning.

By default the *online* evaluation (between training epochs) is disabled (it can be enabled setting the ```eval_startegy="epoch"```), i.e. it doesn't produce the detailed evaluation diagrams with the evolution of metrics over training epochs for computational and speed reasons. Only the final evaluation metrics are reported for each run. 

Furthermore, it is designed for distributed training and uses the ```notebook_launcher()``` function. Don't forget to set the number of processes. Also an appropriate ```accelerate``` config file must be present (see [here](https://huggingface.co/docs/accelerate/en/package_reference/cli) about how to create one).

Note that for a large number of runs (i.e. many different config files) the output notebook becomes "heavy" due to long cell outputs, but in any case it is not of great use, since everything is saved at seperate *run* folders with the following structure:

**run-{x}/** <br>
&emsp;├── log_history.json <br>
&emsp;├── training_args.json <br>
&emsp;└── **evaluation/** <br>
&emsp;&emsp;&emsp;├── forget_{i}.csv <br>
&emsp;&emsp;&emsp;├── retain_{i}.csv <br>
&emsp;&emsp;&emsp;├── mmlu.json <br>
&emsp;&emsp;&emsp;├── evaluation_results.json <br>
&emsp;&emsp;&emsp;├── **mia results/** <br>
&emsp;&emsp;&emsp; |&emsp;&ensp;├── member_{i}.csv <br>
&emsp;&emsp;&emsp; |&emsp;&ensp;└── nonmember_{i}.csv <br>
&emsp;&emsp;&emsp;└── **qualitative/** <br> 
&emsp;&emsp;&emsp;&emsp;&emsp;├── forget_samples.csv <br>
&emsp;&emsp;&emsp;&emsp;&emsp;├── retain_samples.csv <br>
&emsp;&emsp;&emsp;&emsp;&emsp;├── general_questions.csv <br>
&emsp;&emsp;&emsp;&emsp;&emsp;└── summary.xlsx <br>

Finally, a summary of all runs (run_summary.xlsx) is created with the values of the hyperparameters for every run along with the correspondng final metrics (MIA final score, Task Aggregate, MMLU score and Final score).

## Imports

In [1]:
import warnings
import torch
import json
import time
import glob
import os
import pandas as pd

from peft import LoraConfig, get_peft_model, TaskType
from huggingface_hub import snapshot_download
from accelerate import notebook_launcher, Accelerator
from pathlib import Path
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)
from utils.data import DatasetProcessor
from utils.utils import (
    make_compute_metrics,
    preprocess_logits_for_metrics,
    print_number_of_trainable_model_parameters,
    print_gpu_memory,
    plot_metrics,
    plot_training_stats
)
from utils.evaluation import (
    QualitativeEvaluation,
    QuantitativeEvaluation,
    MMLU
)
from methods import AscentPlusDescentDataCollator, SequentialUnlearning

warnings.filterwarnings('ignore')

## Load model and tokenizer

In [4]:
# Load the desired model version

# 7B parameter model
model_repo_id = "llmunlearningsemeval2025organization/olmo-finetuned-semeval25-unlearning"
tokenizer_path = "allenai/OLMo-7B-0724-Instruct-hf"

# 1B parameter model
#model_repo_id = "llmunlearningsemeval2025organization/olmo-1B-model-semeval25-unlearning"
#tokenizer_path = "allenai/OLMo-1B-0724-hf"

snapshot_download(repo_id=model_repo_id, local_dir='pretrained_model')

## Training function

In [ ]:
# MMLU topics (complete list with 57 subjects)

topics = ['abstract_algebra',
          'anatomy',
          'astronomy',
          'business_ethics',
          'clinical_knowledge',
          'college_biology',
          'college_chemistry',
          'college_computer_science',
          'college_mathematics',
          'college_medicine',
          'college_physics',
          'computer_security',
          'conceptual_physics',
          'econometrics',
          'electrical_engineering',
          'elementary_mathematics',
          'formal_logic',
          'global_facts',
          'high_school_biology',
          'high_school_chemistry',
          'high_school_computer_science',
          'high_school_european_history',
          'high_school_geography',
          'high_school_government_and_politics',
          'high_school_macroeconomics',
          'high_school_mathematics',
          'high_school_microeconomics',
          'high_school_physics',
          'high_school_psychology',
          'high_school_statistics',
          'high_school_us_history',
          'high_school_world_history',
          'human_aging',
          'human_sexuality',
          'international_law',
          'jurisprudence',
          'logical_fallacies',
          'machine_learning',
          'management',
          'marketing',
          'medical_genetics',
          'miscellaneous',
          'moral_disputes',
          'moral_scenarios',
          'nutrition',
          'philosophy',
          'prehistory',
          'professional_accounting',
          'professional_law',
          'professional_medicine',
          'professional_psychology',
          'public_relations',
          'security_studies',
          'sociology',
          'us_foreign_policy',
          'virology',
          'world_religions']

In [6]:
def unlearning(args: dict, output_dir: str):
    import transformers
    transformers.logging.set_verbosity_error()
    transformers.logging.disable_progress_bar()

    accelerator = Accelerator()
    
    start = time.time()
    
    # Load the model
    model = AutoModelForCausalLM.from_pretrained('pretrained_model', torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Prepare the data
    processor = DatasetProcessor(data_dir='semeval25-unlearning-data/data', tokenizer=tokenizer)
    dataset = processor(split=args["general"]["split"], task='all', split_tasks=False, split_retain=False)
    
    # Define the data collator
    data_collator = AscentPlusDescentDataCollator(tokenizer=tokenizer, padding='longest', pad_to_multiple_of=8)

    # Prepare the model
    if args["model_params"]["apply_lora"]:
        lora_config = LoraConfig(
            r=args["model_params"]["lora_r"], 
            lora_alpha=args["model_params"]["lora_alpha"],
            target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj"],
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        model = get_peft_model(model, lora_config)
    elif args["model_params"]["train_last_k"]:
        k = args["model_params"]["k"]
        total_layers = len(model.model.layers)
        
        # Freeze all but the last k layers
        for i, layer in enumerate(model.model.layers):
            if i < total_layers - k:  # Freeze these layers
                for param in layer.parameters():
                    param.requires_grad = False
            else:  # Keep these layers trainable
                for param in layer.parameters():
                    param.requires_grad = True

        # Freeze embeddings
        for param in model.model.embed_tokens.parameters():
            param.requires_grad = False

        # Keep lm_head parameters frozen
        for param in model.lm_head.parameters():
            param.requires_grad = False

    print(print_number_of_trainable_model_parameters(model))

    pretrain_model = AutoModelForCausalLM.from_pretrained('pretrained_model', torch_dtype=torch.bfloat16) if args["general"]["retain_loss"] == "KL" else None

    # Training setup
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=args["training_args"]["per_device_batch_size"],
        per_device_eval_batch_size=32,
        gradient_accumulation_steps=args["training_args"]["gradient_accumulation_steps"],
        eval_accumulation_steps=1,
        learning_rate=args["training_args"]["learning_rate"],
        num_train_epochs=args["training_args"]["num_epochs"],
        logging_steps=40,
        ddp_find_unused_parameters=False,
        save_strategy="no",
        eval_strategy="no",
        bf16=True,
        report_to ="none",
        remove_unused_columns=False
    )
    
    trainer = SequentialUnlearning(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        training_args=training_args,
        forget_dataset=dataset['forget'],
        retain_dataset=dataset['retain'],
        compute_metrics=make_compute_metrics(model, tokenizer, max_samples=16),
        preprocess_logits_for_metrics=preprocess_logits_for_metrics,
        sequential=args["general"]["sequential"],
        chunk_size=args["general"]["chunk_size"],
        positive_ratio=args["general"]["positive_ratio"],
        positive_factor=args["general"]["positive_factor"],
        retain_loss=args["general"]["retain_loss"],
        pretrain_model=pretrain_model
    )

    # Train
    trainer.train(split_retain=args["general"]["split_retain"])

    accelerator.wait_for_everyone()

    if accelerator.is_main_process:    
        # Save training stats
        summary = trainer.save_summary(output_dir)
        print(f"\nTotal training time: {summary['total_runtime']}")
        print(f"Total FLOPS: {summary['total_flos']}")

        # Optionally, plot metrics if evaluation is enabled (eval_strategy="epoch")
        #plot_metrics(summary["log_history"], output_dir)
        #plot_training_stats(summary["log_history"])
    
        # Save the unlearned model for final evaluation
        if args["model_params"]["apply_lora"]:
            model.merge_and_unload()
            model.model.save_pretrained("unlearned_model")
        else:
            model.save_pretrained("unlearned_model")
            
        tokenizer.save_pretrained("unlearned_model")
    
        # Print total time
        end = time.time()
        print("\nTotal time:", end-start)

In [ ]:
def evaluate(args: dict, output_dir: str):
    mmlu_start = time.time()
    mmlu = MMLU(topics, seed=42)
    mmlu.run(model_path="unlearned_model", mmlu_metrics_file_path=f"{output_dir}/evaluation/mmlu.json")

    print("MMLU time: ", time.time()-mmlu_start)

    # Quantitative Evaluation
    evaluation_args = {
        "seed": 42,
        "debug": False,
        "keep_files": True,
        "max_new_tokens": 256,
        "compute_metrics_only": False,
        "batch_size": 8,
        "mia_data_path": "semeval25-unlearning-data/mia_data/",
        "split": args["general"]["split"],
        "data_path": "semeval25-unlearning-data/data/",
        "checkpoint_path": "unlearned_model",
        "output_dir": f"{output_dir}/evaluation",
        "mmlu_metrics_file_path": f"{output_dir}/evaluation/mmlu.json"
    }
    quantitative_eval = QuantitativeEvaluation(evaluation_args)
    quantitative_eval.run()

    torch.cuda.empty_cache()

    # Qualitative evaluation
    qualitative_eval = QualitativeEvaluation(
        checkpoint_path="unlearned_model",
        path_to_predictions=f'{output_dir}/evaluation',
        path_to_gqa='general_questions.json',
        output_dir=f'{output_dir}/evaluation/qualitative',
        n_samples=5
    )
    qualitative_eval.run()

## Launch multi-gpu training

In [2]:
# Set the number of processes (GPUs) to be used
# 1 <= NUM_PROCESSES <= N_GPUs

NUM_PROCESSES = 8

In [ ]:
config_files = glob.glob("configs/*.json")
config_files = sorted(config_files, key=lambda x: int(x.split('_')[-1].split(".")[0]))

#output_dir = "/opt/ml/output/data"
output_dir = "outputs"

for i, path in enumerate(config_files):
    # Create the output folder for each run    
    os.makedirs(f"{output_dir}/run-{i}", exist_ok=True)
    run_dir = f"{output_dir}/run-{i}"

    # Load the training arguments from the json file
    with open(path, 'r') as f:
        config = json.load(f)
    
    # Store the training arguments in the output file for future reference
    with open(f"{run_dir}/training_args.json", "w") as f:
        f.write(json.dumps(config, indent=4))
    
    print(f"\nRun {i+1} - Training with the following arguments:\n\n{json.dumps(config, indent=4)}")
 
    # Launch distributed training
    try:
        notebook_launcher(unlearning, (config, run_dir), num_processes=NUM_PROCESSES)
        print("\nTraining completed. Starting the evaluation...")
        notebook_launcher(evaluate, (config, run_dir), num_processes=NUM_PROCESSES)
        print("\nEvaluation completed.")
    except Exception as e:
        print(f"Error during run-{i}: {e}")
        continue

In [ ]:
def generate_summary(output_dir: str):
    files = Path(output_dir).glob("run-*")
    
    # Sort files numerically based on the number after 'run-'
    files = sorted(files, key=lambda x: int(str(x).split('-')[-1]))
    
    data = []
    
    for file in files:
        training_args_path = file / "training_args.json"
        evaluation_results_path = file / "evaluation/evaluation_results.json"

        # Skip if training_args.json doesn't exist
        if not training_args_path.exists():
            print(f"{training_args_path} is missing. Skipping this run.")
            continue
            
        try:
            # Load training arguments
            with training_args_path.open("r") as f1:
                args = json.load(f1)
            
            # Initialize values for evaluation metrics
            args["results"] = {}
            args["results"]["mia_final_score"] = None
            args["results"]["task_aggregate"] = None
            args["results"]["mmlu_average"] = None
            args["results"]["aggregate_score"] = None
            
            # Attempt to load evaluation results
            if evaluation_results_path.exists():
                with evaluation_results_path.open("r") as f2:
                    eval = json.load(f2)
                    args["results"]["mia_final_score"] = eval.get("mia_final_score", None)
                    args["results"]["task_aggregate"] = eval.get("harmonic-mean-task-aggregate", None)
                    args["results"]["mmlu_average"] = eval.get("mmlu_average", None)
                    args["results"]["aggregate_score"] = eval.get("aggregate-score", None)
            else:
                print(f"{evaluation_results_path} is missing. Using None for evaluation metrics.")

            # Desired key order
            desired_order = ["results", "general", "training_args", "model_params"]
    
            # Create a new dictionary with the desired order
            args = {key: args[key] for key in desired_order}
            
            data.append(args)
        except Exception as e:
            print(f"Error processing {file}: {e}")
            continue

    if not data:
        print("No valid data found. Exiting...")
        return 
        
    # Flatten each dictionary and collect them
    flattened_data = [pd.json_normalize(d, sep='.') for d in data]
    
    # Combine all flattened dictionaries into a single DataFrame
    combined_df = pd.concat(flattened_data, ignore_index=True)
    
    # Convert flattened column names into MultiIndex
    multi_index = pd.MultiIndex.from_tuples(
        [tuple(key.split('.')) for key in combined_df.columns],
        names=["Level_1", "Level_2"] 
    )
    
    # Assign MultiIndex to the columns of the DataFrame
    combined_df.columns = multi_index
    
    combined_df.to_csv(f"{output_dir}/run_summary.csv", index=False)
    combined_df.to_excel(f"{output_dir}/run_summary.xlsx")

In [ ]:
generate_summary(output_dir)